In [1]:
#Importing dependencies/libraries

from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
import numpy as np

In [2]:
#Changing approach. Instead of going through several city/locations in a for loop, we will scrape per city/location.
#Scraping per city/location saves allows us to use the data as soon as its scraped.

# #ON
# location_on = ["greater-sudbury", "hamilton", "kingston", "london", "niagara-falls",
#             "ottawa", "toronto", "windsor", "kitchener", "peterborough", 
#             "sault-ste-marie", "thunder-bay"]

# #AB
# location_ab = ["edmonton", "calgary", "lethbridge", "red-deer"]

# #NS
# location_ns = ["halifax"]

# #QC
# location_qc = ["montreal", "sherbrooke", "trois-rivieres", "quebec"]

# #BC
# location_bc = ["vancouver", "victoria"]

# #MB
# location_mb = ["winnipeg"]

#------ADJUST LOCATION VARIABLE------
location_nu = ["iqaluit"]
prov = ['nu'] # --------> This format does not make sense especially since this variable is not used in a for loop, but the code for front page scraping will stop working if the format is changed.

#Yelp shows 10 restaurants (not including sponsored ones) per page. 
#The web page can be changed by manipulating the number at after the'start=' line of the url.
pages = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230]

In [3]:
#Scrape links for each restaurant from the yelp's front page

#Empty lists to hold the values
restaurant_links = []

#Webscraping for restaurant links
browser = Browser('chrome')

#This for loop changes the page on the url
#------ADJUST LOCATION VARIABLE------
for loc in location_nu:
    
    #This for loop changes the location of the url
    for page in pages:  
        
        url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc={loc}%2C+{prov}&start={page}'
        browser.visit(url)
        html = browser.html
        soup_obj = soup(html, 'html.parser')
        
        res_info = []
        res_info = soup_obj.find_all("div", class_ = "businessName__09f24__EYSZE display--inline-block__09f24__fEDiJ border-color--default__09f24__NPAKY")
        time.sleep(3)
        
        for info in res_info:
            
            try:
                target = info.a['href']
                restaurant_links.append(target)

            except:
                restaurant_links.append("No data or change code")    
                
browser.quit()

In [4]:
#Visually checking scraping results
restaurant_links

['/biz/gallery-fine-dining-room-iqaluit?osq=Restaurants',
 '/biz/yummy-shawarma-iqaluit?osq=Restaurants',
 '/biz/the-frob-kitchen-and-eatery-iqaluit?osq=Restaurants',
 '/biz/the-snack-iqaluit-2?osq=Restaurants',
 '/biz/black-heart-cafe-iqaluit?osq=Restaurants',
 '/biz/storehouse-bar-and-grill-iqaluit?osq=Restaurants',
 '/biz/kfc-iqaluit?osq=Restaurants',
 '/biz/frobisher-racquet-club-iqaluit?osq=Restaurants',
 '/biz/royal-canadian-legion-iqaluit?osq=Restaurants']

In [5]:
#Visually checking scraping results
len(restaurant_links)

9

In [6]:
#Extracting address from each link, using the link in the list restaurant_links

#Setting up a counter to keep track of progress; this code takes about ~30 mins to run
counter = 1

#Setting up empty lists
names = []
# total_reviews = []
# all_information = []
restaurant_address = []

#Extracting restaurant name and address from restaurant's business page
browser = Browser('chrome')

for link in restaurant_links:
       
    #Visiting the resturant's Yelp page 
    base_url = "https://www.yelp.com"
    res_url = f"https://www.yelp.com{link}"

    browser.visit(res_url)

    html = browser.html

    soup_obj = soup(html, 'html.parser')
    time.sleep(3)
    
    #Extracting name
    name = soup_obj.find('h1', class_ = "css-1se8maq")
    if name:
        data = name.text
        names.append(data)

    else:
        names.append("No data or change code")


#     reviews = soup_obj.find('a', class_ = "css-19v1rkv")
#     if reviews:
#         data2 = reviews.text
#         total_reviews.append(data2)
#         print(data2)
#         print("------")
#     else:
#         print("No data or change code")

#     information = soup_obj.find_all('span', class_ = "css-1fdy0l5")
#     if information:
#         for info in information:
#             all_information.append(info.text)
#             print(info.text)

#     else:
#         print("No data or change code")
    
    #Extracting address
    address = soup_obj.find('p', class_ = "css-qyp8bo")

    if address:
        data3 = address.text
        restaurant_address.append(data3)

    else:
        restaurant_address.append("No data or change code")
        
    print(f'Link # {counter}')
    counter += 1

browser.quit()

Link # 1
Link # 2
Link # 3
Link # 4
Link # 5
Link # 6
Link # 7
Link # 8
Link # 9


In [7]:
#Visually checking scraping results
restaurant_address

['No data or change code',
 '1089 Mivvik Street Iqaluit, NU X0A 0H0 Canada',
 'Astro Hill Complex Iqaluit, NU X0A 0H0 Canada',
 '163 Nipisa Street Iqaluit, NU X0A 0H0 Canada',
 '1405 Masak Court Iqaluit, NU X0A 0H0 Canada',
 'No data or change code',
 '157 Nipisa Street Iqaluit, NU X0A 0H0 Canada',
 'Building 902 Iqaluit, NU X0A Canada',
 'Iqaluit, NU Canada']

In [8]:
#Visually checking scraping results
len(restaurant_address)

9

In [9]:
#Converting and joining restaurant name and address scraped from business page
test_df = pd.DataFrame({
    "restaurant_name" : names,
    "address" : restaurant_address
})
test_df

,restaurant_name,address
0,Gallery Fine Dining Room,No data or change code
1,Yummy Shawarma,"1089 Mivvik Street Iqaluit, NU X0A 0H0 Canada"
2,The Frob Kitchen & Eatery,"Astro Hill Complex Iqaluit, NU X0A 0H0 Canada"
3,The Snack,"163 Nipisa Street Iqaluit, NU X0A 0H0 Canada"
4,Black Heart Cafe,"1405 Masak Court Iqaluit, NU X0A 0H0 Canada"
5,Storehouse Bar & Grill,No data or change code
6,KFC,"157 Nipisa Street Iqaluit, NU X0A 0H0 Canada"
7,Frobisher Racquet Club,"Building 902 Iqaluit, NU X0A Canada"
8,Royal Canadian Legion,"Iqaluit, NU Canada"


In [10]:
#Scraping information from Yelp's front page

#Empty lists to hold the values
names_fps = []
price_range = []
ratings = []
total_reviews = []
category = []
location = []
province = []

#Web scraping
browser = Browser('chrome')

#This for loop changes the page on the url
#------ADJUST LOCATION VARIABLE------
for loc in location_nu:
    
    #This for loop changes the location of the url
    for page in pages:  
        
        url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc={loc}%2C+{prov}&start={page}'
        browser.visit(url)
        html = browser.html
        soup_obj = soup(html, 'html.parser')      
        
        #Scraping the div that contains the target elements
        res_info = []    
        res_info = soup_obj.find_all("div", class_ = "container__09f24__mpR8_ hoverable__09f24__wQ_on border-color--default__09f24__NPAKY")
        time.sleep(3)
        
        #Extracting the target elements
        for info in res_info:
            #Extracting restaurant name
            try:
                name = info.find("a", class_ = "css-19v1rkv").text
                names_fps.append(name)
            except:
                names_fps.append("No data or change the code")
              
            #Extracting price range
            if info.find("span", {"class":"priceRange__09f24__mmOuH css-chan6m"}):
                prc_range = info.find("span", {"class":"priceRange__09f24__mmOuH css-chan6m"}).text
                price_range.append(prc_range)
            elif info.find("span", {"class":"priceRange__09f24__mmOuH css-1s7bx9e"}):
                prc_range = info.find("span", {"class":"priceRange__09f24__mmOuH css-1s7bx9e"}).text
                price_range.append(prc_range)
            else:
                price_range.append("No data or change the code")
            
            #Extracting ratings
            try:
                rati = info.find("span", {"class":"css-gutk1c"}).text
                ratings.append(rati)
            except:
                ratings.append("No data or change the code")
            
            #Extracting total number of reviews
            #Yelp seems to have a different class for reviews in Montreal's restaurant front page, so code is adjusted
            try: #---------------------> OG code
                revs = info.find("span", {"class":"css-chan6m"}).text # ---------> Option 1
#                 revs = info.find("span", class_ = "css-chan6m").text # --------> Option 2, changed the code for front page scraping St. John's, NB. Might be because front page is different (no sponsored restaurants).
                total_reviews.append(revs)
            except:
                total_reviews.append("No data or change the code")
            #--------------> option 2, Montreal    
#             if info.find("span", {"class":"css-8xcil9"}):
#                 revs = info.find("span", {"class":"css-8xcil9"}).text
#                 total_reviews.append(revs)
#             elif info.find("span", {"class":"css-chan6m"}):
#                 revs = info.find("span", {"class":"css-chan6m"}).text
#                 total_reviews.append(revs)
#             else:
#                 total_reviews.append("No data or change the code")
            
            
            #Extracting category
            try:
                categ = info.find("span", {"class":"css-11bijt4"}).text
                category.append(categ)
            except:
                category.append("No data or change the code")
            
            #Assigning location and province.
            #------ADJUST PROVINCE TO APPEND------
            location.append(loc)
            province.append("NU")

browser.quit()

In [11]:
#Visually checking scraping results
total_reviews

['(4 reviews)',
 '(1 review)',
 '(1 review)',
 '(1 review)',
 '(3 reviews)',
 '(8 reviews)',
 '',
 '',
 '']

In [12]:
#Visually checking scraping results
len(total_reviews)

9

In [13]:
#Converting and joining data scraped from front page
test_df2 = pd.DataFrame({
    "restaurant_name" : names_fps,
    "price_range" : price_range,
    "rating" : ratings,
    "total_reviews" : total_reviews,
    "category" : category,
    "location" : location,
    "province" : province
})

test_df2

,restaurant_name,price_range,rating,total_reviews,category,location,province
0,Gallery Fine Dining Room,$$$,4.0,(4 reviews),Canadian (New),iqaluit,NU
1,Yummy Shawarma,No data or change the code,4.0,(1 review),Mediterranean,iqaluit,NU
2,The Frob Kitchen & Eatery,No data or change the code,5.0,(1 review),Diners,iqaluit,NU
3,The Snack,No data or change the code,1.0,(1 review),Burgers,iqaluit,NU
4,Black Heart Cafe,No data or change the code,4.7,(3 reviews),Cafes,iqaluit,NU
5,Storehouse Bar & Grill,$$,2.6,(8 reviews),Canadian (New),iqaluit,NU
6,KFC,No data or change the code,No data or change the code,,Chicken Shop,iqaluit,NU
7,Frobisher Racquet Club,No data or change the code,No data or change the code,,Gyms,iqaluit,NU
8,Royal Canadian Legion,No data or change the code,No data or change the code,,American (Traditional),iqaluit,NU


In [14]:
#Merging business page and front page dataframe
merged_df = pd.merge(test_df2, test_df, on="restaurant_name", how = "left")
merged_df

,restaurant_name,price_range,rating,total_reviews,category,location,province,address
0,Gallery Fine Dining Room,$$$,4.0,(4 reviews),Canadian (New),iqaluit,NU,No data or change code
1,Yummy Shawarma,No data or change the code,4.0,(1 review),Mediterranean,iqaluit,NU,"1089 Mivvik Street Iqaluit, NU X0A 0H0 Canada"
2,The Frob Kitchen & Eatery,No data or change the code,5.0,(1 review),Diners,iqaluit,NU,"Astro Hill Complex Iqaluit, NU X0A 0H0 Canada"
3,The Snack,No data or change the code,1.0,(1 review),Burgers,iqaluit,NU,"163 Nipisa Street Iqaluit, NU X0A 0H0 Canada"
4,Black Heart Cafe,No data or change the code,4.7,(3 reviews),Cafes,iqaluit,NU,"1405 Masak Court Iqaluit, NU X0A 0H0 Canada"
5,Storehouse Bar & Grill,$$,2.6,(8 reviews),Canadian (New),iqaluit,NU,No data or change code
6,KFC,No data or change the code,No data or change the code,,Chicken Shop,iqaluit,NU,"157 Nipisa Street Iqaluit, NU X0A 0H0 Canada"
7,Frobisher Racquet Club,No data or change the code,No data or change the code,,Gyms,iqaluit,NU,"Building 902 Iqaluit, NU X0A Canada"
8,Royal Canadian Legion,No data or change the code,No data or change the code,,American (Traditional),iqaluit,NU,"Iqaluit, NU Canada"


In [15]:
#Cleaning merged dataframe
merged_df.drop_duplicates()
merged_df.replace('No data or change the code', np.nan, inplace = True)
merged_df.dropna(inplace = True)
merged_df = merged_df.reset_index(drop = True)
merged_df['total_reviews'] = merged_df['total_reviews'].str.replace(r'[\(\) reviews]', '', regex=True)

merged_df

,restaurant_name,price_range,rating,total_reviews,category,location,province,address
0,Gallery Fine Dining Room,$$$,4.0,4,Canadian (New),iqaluit,NU,No data or change code
1,Storehouse Bar & Grill,$$,2.6,8,Canadian (New),iqaluit,NU,No data or change code


In [16]:
#Visually checking dataframe
data_checking = merged_df['restaurant_name'].value_counts()
data_checking.head(10)

Gallery Fine Dining Room    1
Storehouse Bar & Grill      1
Name: restaurant_name, dtype: int64

In [17]:
#Visually checking dataframe
#------ADJUST VALUE TO BE CHECKED------
merged_df[merged_df.eq('').any(axis=1)]


,restaurant_name,price_range,rating,total_reviews,category,location,province,address


In [ ]:
#Further cleaning dataframe, each restaurant to be dropped were visually inspected through their Yelp business page
merged_df.drop([
               ], inplace = True)
#Did not drop anything

In [18]:
#Visually checking dataframe
data_checking2 = merged_df['restaurant_name'].value_counts()
data_checking2.head(10)

Gallery Fine Dining Room    1
Storehouse Bar & Grill      1
Name: restaurant_name, dtype: int64

In [19]:
#Making an output csv
#------ADJUST CSV NAME------
merged_df.to_csv("yelp_iqaluit_nu_complete.csv", index = False)